In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.2`
import $ivy.`org.apache.spark::spark-core:3.2.2`
import $ivy.`org.apache.spark::spark-mllib:3.2.2`

import $ivy.$                                  

import $ivy.$                                   

import $ivy.$                                    

In [25]:
import org.apache.log4j.{Level, Logger}
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.regression._
import org.apache.spark.mllib.classification.{SVMModel,
SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.LabeledPoint
import org.apache.spark.ml.linalg.Vectors
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext

Creating SparkSession


Spark UI

import org.apache.log4j.{Level, Logger}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.mllib.linalg._

import org.apache.spark.mllib.regression._

import org.apache.spark.mllib.classification.{SVMModel,
SVMWithSGD}

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.util.MLUtils

import org.apache.spark.ml.feature.LabeledPoint

import org.apache.spark.ml.linalg.Vectors

spark: SparkSession = org.apache.spark.sql.SparkSession@4e9643ad
defined function sc

In [3]:
val data= sc.textFile("/home/jb-admin/data/iris.csv")

data: org.apache.spark.rdd.RDD[String] = /home/jb-admin/data/iris.csv MapPartitionsRDD[1] at textFile at cmd2.sc:1

In [4]:
data.take(5)

take at cmd3.sc:1

1 / 1

res3: Array[String] = Array(
  "sepal_length,sepal_width,petal_width,petal_width,species",
  "5.1,3.5,1.4,0.2,Iris-setosa",
  "4.9,3,1.4,0.2,Iris-setosa",
  "4.7,3.2,1.3,0.2,Iris-setosa",
  "4.6,3.1,1.5,0.2,Iris-setosa"
)

In [5]:
// data
val datafile= sc.textFile("/home/jb-admin/data/iris.csv").filter(!_.contains("length"))
datafile.take(5)

take at cmd4.sc:2

1 / 1

datafile: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at filter at cmd4.sc:1
res4_1: Array[String] = Array(
  "5.1,3.5,1.4,0.2,Iris-setosa",
  "4.9,3,1.4,0.2,Iris-setosa",
  "4.7,3.2,1.3,0.2,Iris-setosa",
  "4.6,3.1,1.5,0.2,Iris-setosa",
  "5,3.6,1.4,0.2,Iris-setosa"
)

In [6]:
val splits = data.randomSplit(Array(0.6, 0.4), seed = 11L)

splits: Array[org.apache.spark.rdd.RDD[String]] = Array(
  MapPartitionsRDD[5] at randomSplit at cmd5.sc:1,
  MapPartitionsRDD[6] at randomSplit at cmd5.sc:1
)

In [8]:
// run training data
val numIterations = 100

numIterations: Int = 100

In [10]:
def parse(line: String, class0: String)= {
    val ss = line.split(',')
    val label = ss.last match{
        case `class0` => 0.0 
        case _ => 1.0               
    }
    new LabeledPoint(label, Vectors.dense(ss.take(ss.length-1).map(_.toDouble)))
}


defined function parse

In [26]:
// have to use the scal.math because we are using the acorn kernel for jupyter notebooks
def parse2(line: String, class0: String) = {
  val ss = line.split(',')
  val label = ss.last match {
    case `class0` => 0.0
    case _ => 1.0
  }
  val v1 = ss.take(ss.length - 1).map(_.toDouble)
  val v2 = v1.map(x => x / scala.math.sqrt(3))
  val v3 = v1.map(x => scala.math.pow(x, 2) / scala.math.sqrt(27))
  val v4 = v1.map(x => scala.math.pow(x, 3))

  // Kernel expansion (x*y + 1)^3 => (1, x/sqrt(3), x^2/sqrt(27), x^3)
  val kernel_expansion = Vectors.dense(1.0 +: (v2 ++ v3 ++ v4))

  new LabeledPoint(label, kernel_expansion)
}

defined function parse2

In [27]:
// filter out one class of data
val data_filter1 = datafile.filter(!_.contains("virginica"))
val data_filter2 = datafile.filter(!_.contains("versicolor"))
val data_filter3 = datafile.filter(!_.contains("setosa"))
data_filter1.take(5)
data_filter2.take(5)
data_filter3.take(5)

val data1 = data_filter1.map(parse(_, "Iris-setosa")) //setosa versicolor
val data2 = data_filter3.map(parse(_, "Iris-versicolor")) //versicolor virginica
val data3 = data_filter2.map(parse(_, "Iris-virginica")) //setosa virginica

// versicolor and virginica not separable
// apply kernel

val data4 = data_filter3.map(parse2(_, "Iris-versicolor")) // versicolor virginica parse 2

print("Top rows of data1")
data1.take(5)
data2.take(5)
data3.take(5)
data4.take(5)


take at cmd26.sc:4

1 / 1

take at cmd26.sc:4

1 / 1

take at cmd26.sc:5

...

take at cmd26.sc:21

1 / 1

take at cmd26.sc:21

1 / 1

data_filter1: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1120] at filter at cmd26.sc:1
data_filter2: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1121] at filter at cmd26.sc:2
data_filter3: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[1122] at filter at cmd26.sc:3
res26_3: Array[String] = Array(
  "5.1,3.5,1.4,0.2,Iris-setosa",
  "4.9,3,1.4,0.2,Iris-setosa",
  "4.7,3.2,1.3,0.2,Iris-setosa",
  "4.6,3.1,1.5,0.2,Iris-setosa",
  "5,3.6,1.4,0.2,Iris-setosa"
)
res26_4: Array[String] = Array(
  "5.1,3.5,1.4,0.2,Iris-setosa",
  "4.9,3,1.4,0.2,Iris-setosa",
  "4.7,3.2,1.3,0.2,Iris-setosa",
  "4.6,3.1,1.5,0.2,Iris-setosa",
  "5,3.6,1.4,0.2,Iris-setosa"
)
res26_5: Array[String] = Array(
  "7,3.2,4.7,1.4,Iris-versicolor",
  "6.4,3.2,4.5,1.5,Iris-versicolor",
  "6.9,3.1,4.9,1.5,Iris-versicolor",
  "5.5,2.3,4,1.3,Iris-versicolor",
  "6.5,2.8,4.6,1.5,Iris-versicolor"
)
data1: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1123] at map at cmd26.sc:8
data2: org.apache.

In [28]:
// run the model with setosa and versicolor

// SVM example
//val splits = data.randomSplit(Array(0.6, 0.4), seed=123)
val training = data1
val test = data1

// run training
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

model.clearThreshold()

val scoreAndLabels = test.map { point =>
    val score = model.predict(point.features)
    (score, point.label)
}


first at GeneralizedLinearAlgorithm.scala:200

1 / 1

first at GeneralizedLinearAlgorithm.scala:200

1 / 1

count at DataValidators.scala:38

...

treeAggregate at GradientDescent.scala:246

2 / 2

treeAggregate at GradientDescent.scala:246

2 / 2

training: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1123] at map at cmd26.sc:8
test: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1123] at map at cmd26.sc:8
numIterations: Int = 100
model: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
res27_4: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
scoreAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1332] at map at cmd27.sc:10

In [29]:

val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC1 = metrics.areaUnderROC()

map at BinaryClassificationMetrics.scala:48

2 / 2

map at BinaryClassificationMetrics.scala:48

2 / 2

sortByKey at BinaryClassificationMetrics.scala:189

...

collect at AreaUnderCurve.scala:44

2 / 2

collect at AreaUnderCurve.scala:44

2 / 2

metrics: BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@4c8934f8
auROC1: Double = 1.0

In [30]:
// run the model with versicolor and virginica

// SVM example
//val splits = data.randomSplit(Array(0.6, 0.4), seed=123)
val training = data2
val test = data2

// run training
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

model.clearThreshold()

val scoreAndLabels = test.map { point =>
    val score = model.predict(point.features)
    (score, point.label)
}


first at GeneralizedLinearAlgorithm.scala:200

1 / 1

first at GeneralizedLinearAlgorithm.scala:200

1 / 1

count at DataValidators.scala:38

...

treeAggregate at GradientDescent.scala:246

2 / 2

treeAggregate at GradientDescent.scala:246

2 / 2

training: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1124] at map at cmd26.sc:9
test: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1124] at map at cmd26.sc:9
numIterations: Int = 100
model: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
res29_4: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
scoreAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1550] at map at cmd29.sc:10

In [31]:
// performace metrics of sample model

val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC2 = metrics.areaUnderROC()

map at BinaryClassificationMetrics.scala:48

2 / 2

map at BinaryClassificationMetrics.scala:48

2 / 2

sortByKey at BinaryClassificationMetrics.scala:189

...

collect at AreaUnderCurve.scala:44

2 / 2

collect at AreaUnderCurve.scala:44

2 / 2

metrics: BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@a0113bd
auROC2: Double = 0.9951999999999999

In [32]:
// run the model with setosa and virginica

// SVM example
//val splits = data.randomSplit(Array(0.6, 0.4), seed=123)
val training = data3
val test = data3

// run training
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

model.clearThreshold()

val scoreAndLabels = test.map { point =>
    val score = model.predict(point.features)
    (score, point.label)
}

first at GeneralizedLinearAlgorithm.scala:200

1 / 1

first at GeneralizedLinearAlgorithm.scala:200

1 / 1

count at DataValidators.scala:38

...

treeAggregate at GradientDescent.scala:246

2 / 2

treeAggregate at GradientDescent.scala:246

2 / 2

training: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1125] at map at cmd26.sc:10
test: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1125] at map at cmd26.sc:10
numIterations: Int = 100
model: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
res31_4: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 4, numClasses = 2, threshold = None
scoreAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1768] at map at cmd31.sc:10

In [33]:
// performace metrics of sample model

val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC3 = metrics.areaUnderROC()

map at BinaryClassificationMetrics.scala:48

2 / 2

map at BinaryClassificationMetrics.scala:48

2 / 2

sortByKey at BinaryClassificationMetrics.scala:189

...

collect at AreaUnderCurve.scala:44

2 / 2

collect at AreaUnderCurve.scala:44

2 / 2

metrics: BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@66a29217
auROC3: Double = 1.0

In [34]:
// run the model with versicolor and virginica parse2

// SVM example
//val splits = data.randomSplit(Array(0.6, 0.4), seed=123)
val training = data4
val test = data4

// run training
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

model.clearThreshold()

val scoreAndLabels = test.map { point =>
    val score = model.predict(point.features)
    (score, point.label)
}


first at GeneralizedLinearAlgorithm.scala:200

1 / 1

first at GeneralizedLinearAlgorithm.scala:200

1 / 1

count at DataValidators.scala:38

...

treeAggregate at GradientDescent.scala:246

2 / 2

treeAggregate at GradientDescent.scala:246

2 / 2

training: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1126] at map at cmd26.sc:15
test: org.apache.spark.rdd.RDD[LabeledPoint] = MapPartitionsRDD[1126] at map at cmd26.sc:15
numIterations: Int = 100
model: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 13, numClasses = 2, threshold = None
res33_4: SVMModel = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 13, numClasses = 2, threshold = None
scoreAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1986] at map at cmd33.sc:10

In [35]:
// performace metrics of sample model

val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC4 = metrics.areaUnderROC()

map at BinaryClassificationMetrics.scala:48

2 / 2

map at BinaryClassificationMetrics.scala:48

2 / 2

sortByKey at BinaryClassificationMetrics.scala:189

...

collect at AreaUnderCurve.scala:44

2 / 2

collect at AreaUnderCurve.scala:44

2 / 2

metrics: BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@b84dc21
auROC4: Double = 0.9691999999999998

In [36]:

println()
println(s"Area under ROC for setosa and versicolor = $auROC1")
println(s"Area under ROC for versicolor and virginica = $auROC2")
println(s"Area under ROC for setosa and virginica = $auROC3")
println()
println(s"Area under ROC for versicolor and virginica with kernal expansion = $auROC4")
println()
println("Program complete")



Area under ROC for setosa and versicolor = 1.0
Area under ROC for versicolor and virginica = 0.9951999999999999
Area under ROC for setosa and virginica = 1.0

Area under ROC for versicolor and virginica with kernal expansion = 0.9691999999999998

Program complete
